In [29]:
import numpy as np
import time
import ast
from subprocess import Popen, PIPE
from pathlib import Path

In [22]:
root = Path("../scripts/smac")
output = root.joinpath("smac3-output_3_20")
dataset = root.joinpath("3_20/train")
cmd_basic = root.joinpath("multiobj")

seeds = [1234, 2, 221190, 317, 777]

wt_lst = ['max_weight',
    'min_weight',
    'max_avg_value',
    'min_avg_value',
    'max_max_value',
    'min_max_value',
    'max_min_value',
    'min_min_value',
    'max_avg_value_by_weight',
    'max_max_value_by_weight']

train_instances = [dataset.joinpath(f"kp_7_3_20_{i}.dat") for i in range(250)]
test_instances = [dataset.joinpath(f"kp_7_3_20_{i}.dat") for i in range(250, 500)]

In [25]:
def find_run_time(instances, wt_str):
    total_time = 0
    count = 0
    for instance in instances:
        cmd = f"{str(cmd_basic)} {instance} ./sample_weighted.csv 1"
        cmd = cmd + " " + wt_str

        io = Popen(cmd.split(" "), stdout=PIPE, stderr=PIPE)
        (stdout_, stderr_) = io.communicate()

        # print(stdout_, stderr_)
        stdout_str = stdout_.decode('utf-8')
        if len(stdout_str):
            status, result = stdout_str.split(":")            
            if status == "Solved":
                total_time += np.sum(list(map(float, stdout_str.strip().split(',')[-3:])))
                count += 1
                
    return total_time/count

In [28]:
for seed in seeds:
    run_path = output.joinpath(f"run_{seed}")
    traj_path = run_path / "traj.json"
    print(f"Seed {seed}")
    
    best_test_time = 1e6
    best_test_idx = None
    with open(traj_path, "r") as traj:
        for idx, l in enumerate(traj.readlines()[1:]):
            ld = ast.literal_eval(l)
            incumbent = ld["incumbent"]            
            wt_str = " ".join([f'{incumbent[k]}' for k in wt_lst])
            
            train_time = find_run_time(train_instances, wt_str)
            test_time = find_run_time(test_instances, wt_str)
            
            print(f"Incumbent id: {idx+1}, Train time: {train_time}, Test time: {test_time}")
            if test_time < best_test_time:
                best_test_time = test_time
                best_test_idx = idx+1
    print("-------------------------------------------------------------------")
    print(f"Best Incumbent: {best_test_idx}, best test time {best_test_time}")
    print("\n")

Seed 1234
Incumbent id: 1, Train time: 0.02760885599999999, Test time: 0.028309240000000006
Incumbent id: 2, Train time: 0.028073216, Test time: 0.02873656000000001
Incumbent id: 3, Train time: 0.027647456, Test time: 0.028257856000000022
Incumbent id: 4, Train time: 0.028494363999999987, Test time: 0.02936996400000001
Incumbent id: 5, Train time: 0.028036052000000016, Test time: 0.029817487999999986
Incumbent id: 6, Train time: 0.028590300000000003, Test time: 0.02907392799999998
Incumbent id: 7, Train time: 0.027987011999999988, Test time: 0.02863321199999999
Incumbent id: 8, Train time: 0.02835319999999999, Test time: 0.02886603599999999
-------------------------------------------------------------------
Best Incumbent: 3, best test time 0.028257856000000022


Seed 2
Incumbent id: 1, Train time: 0.027197503999999994, Test time: 0.027968375999999986
Incumbent id: 2, Train time: 0.028446816000000017, Test time: 0.02898856400000001
Incumbent id: 3, Train time: 0.02855899999999999, Test

In [8]:
instances = [f"3_20/train/kp_7_3_20_{i}.dat" for i in range(250)]

{"max_avg_value": 0.4321253653800381, "max_avg_value_by_weight": 0.0638920131482687, "max_max_value": 0.17548031682439358, "max_max_value_by_weight": 0.07585762180182958, "max_min_value": 0.73275423425348, "max_weight": 0.822226327737958, "min_avg_value": 0.20912683008655408, "min_max_value": 0.23968137762255143, "min_min_value": 0.5989891421182663, "min_weight": 0.06619521915267598}


wt_dict = {"max_avg_value": 0.4500185954509764, 
           "max_avg_value_by_weight": 0.47900710822884596, 
           "max_max_value": 0.10105274422511748, 
           "max_max_value_by_weight": 0.14390745990629927, 
           "max_min_value": 0.2909215164563521, 
           "max_weight": 0.9182967088730671, 
           "min_avg_value": 0.42199285253811414, 
           "min_max_value": 0.525232096333076, 
           "min_min_value": 0.1087829211786774, 
           "min_weight": 0.9089081579717498}

0.9182967088730671 0.9089081579717498 0.4500185954509764 0.42199285253811414 0.10105274422511748 0.525232096333076 0.2909215164563521 0.1087829211786774 0.47900710822884596 0.14390745990629927


In [1]:
for instance in instances:
    cmd = f"./multiobj {instances} ./sample_weighted.csv 1"
    cmd = cmd + " " + wt_str

    io = Popen(cmd.split(" "), stdout=PIPE, stderr=PIPE)
    (stdout_, stderr_) = io.communicate()

    # print(stdout_, stderr_)
    stdout_str = stdout_.decode('utf-8')

    if len(stdout_str):
        status, result = stdout_str.split(":")
        print(result)
        if status == "Solved":
            print(np.sum(list(map(float, stdout_str.strip().split(',')[-3:]))))


0.9182967088730671
0.9089081579717498
0.4500185954509764
0.42199285253811414
0.10105274422511748
0.525232096333076
0.2909215164563521
0.1087829211786774
0.47900710822884596
0.14390745990629927
